In [12]:
!hostname

DESKTOP-VD1817E


In [ ]:
import subprocess
import sys

# Liste der benötigten Bibliotheken
required_packages = [
    "gymnasium", "numpy", "pandas", "joblib", "scikit-learn", "matplotlib",
    "stable-baselines3", "torch", "torchvision", "torchaudio"
]

# Funktion zum Installieren fehlender Bibliotheken
def install_packages(packages):
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            print(f"📦 Installiere {package} ...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Installiere fehlende Pakete
install_packages(required_packages)

# 🔹 Separate Installation für sb3_contrib
try:
    import sb3_contrib
except ImportError:
    print("📦 Installiere sb3_contrib ...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "stable-baselines3[extra]"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sb3-contrib"])

In [13]:
#%run /home/dhbw/environment.ipynb
%run ../../Environment/environment.ipynb

e:\Studienarbeit_GitHub\Studienarbeit\Agents\PPO
Notebook ausgeführt


In [ ]:
import numpy as np
import pandas as pd
import joblib
import random
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.callbacks import CheckpointCallback
from sklearn.preprocessing import StandardScaler
from stable_baselines3.common.vec_env import DummyVecEnv
import torch

In [14]:
seed = 42
SEED  = seed % (2**32 - 1)
print(f"SEED: {SEED}")

SEED: 42


In [15]:
# -------------------------------
# CSV Datem einlesen
# -------------------------------
dhbw_train_path = "/home/dhbw/2023-2018_stand_data.csv"
desktop_train_path = "../../Transform_data/stand_data/2023-2018_stand_data.csv"
train_data = pd.read_csv(desktop_train_path)
train_data.drop('datetime', axis=1, inplace=True)

dhbw_test_path = "/home/dhbw/2025-2024_stand_data.csv"
desktop_test_path = "../../Transform_data/stand_data/2025-2024_stand_data.csv"
test_data = pd.read_csv(desktop_test_path)
test_data.drop('datetime', axis=1, inplace=True)

if(train_data is not None and test_data is not None):
    print("Daten erfolgreich eingelesen")

Daten erfolgreich eingelesen


In [ ]:
from stable_baselines3.common.vec_env import SubprocVecEnv

def make_env():
    return TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path="/home/dhbw/scaler.pkl",
        default_seed=SEED
    )

n_envs = 8  # Mehr parallele Umgebungen (8, 16 oder sogar 32 testen!)
env = SubprocVecEnv([make_env for _ in range(n_envs)])

In [ ]:
# Test-Umgebung für Evaluation (ohne SubprocVecEnv, da wir nur eine Instanz brauchen)
test_env = TradingEnv(
    data=test_data,  # Oder test_data, falls du separate Test-Daten hast
    initial_cash=10_000,
    window_size=336,
    scaler_path="/home/dhbw/scaler.pkl",
    default_seed=SEED
)

In [ ]:
from stable_baselines3.common.vec_env import VecNormalize

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)
env.training = True  # Sicherstellen, dass Normalisierung aktiv ist
env.device = "cuda"  # WICHTIG: Umgebung auf CUDA setzen

# === Test ===

In [16]:
dhbw_scaler = "/home/dhbw/scaler.pkl"
desktop_scaler = "../../Transform_data/scaler.pkl"

env = TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path=desktop_scaler,
        default_seed=SEED
    )

Seed in the environment: 42


In [22]:
import numpy as np
import gymnasium as gym
from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.utils import get_action_masks
from stable_baselines3.common.vec_env import DummyVecEnv

In [23]:
env = DummyVecEnv([lambda: TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path=desktop_scaler,
        default_seed=SEED
    )()])
model = MaskablePPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000, use_masking=True)

# 🚀 Prediction auf ungesehenen Daten
obs, _ = env.reset()
done = False
while not done:
    action_masks = env.get_action_mask()  # 🔥 Maske für gültige Aktionen holen
    action, _ = model.predict(obs, action_masks=action_masks)  # 🎯 Maske nutzen
    obs, reward, done, _, _ = env.step(action)

env.render(mode="human")

Seed in the environment: 42


TypeError: 'TradingEnv' object is not callable

In [26]:
from sb3_contrib.common.maskable.utils import is_masking_supported

env = TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path=desktop_scaler,
        default_seed=SEED
    )

print("🚀 Unterstützt Masking?:", is_masking_supported(env))  # Sollte `True` sein


Seed in the environment: 42
🚀 Unterstützt Masking?: False


# === Ende ===

In [ ]:
from torch import nn  # Für die Netzwerkarchitektur

# Definiere das neuronale Netz
policy_kwargs = dict(
    net_arch=[dict(pi=[512, 512, 128], vf=[512, 512, 128])],  # Zwei große Layer (512) und ein kleinerer Layer (128)
    activation_fn=nn.ReLU,
)

# Erstelle den PPO-Agenten mit verbesserten Einstellungen
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=0.0001,  # Lernrate
    gamma=0.99,  # Discount-Faktor
    clip_range=0.2,  # PPO-Clip-Parameter
    ent_coef=0.01,  # Entropie-Koeffizient
    n_steps=4_096,  # WICHTIG: Mehr Schritte pro Update → GPU-Auslastung steigt
    batch_size=4_096,  # WICHTIG: Große Batch-Größe → GPU rechnet effizienter
    policy_kwargs=policy_kwargs,
    verbose=1,
    seed=SEED,
    device="cuda",  # Nutzt die GPU!
    #tensorboard_log="./tensorboard_log/"  # Optional: Logging für TensorBoard
)


In [ ]:
print(f"Env läuft auf: {env.device}")  # Sollte CUDA anzeigen
print("Modell läuft auf:", model.device)
print("Model Policy:", model.policy.device)

In [ ]:
print(type(model.policy))
print(hasattr(model.policy, 'to'))

In [ ]:
model.policy.to("cuda")
print("Policy erfolgreich auf GPU gesetzt:", next(model.policy.parameters()).device)

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Optional: Checkpoint Callback, falls du Zwischenspeicherungen möchtest
# checkpoint_callback = CheckpointCallback(save_freq=100, save_path='./logs/', name_prefix='ppo_trading')

# -------------------------------
# Trainings- und Test-Performance (Loss) evaluieren und plotten
# -------------------------------
# Wir unterteilen das Training in mehrere Intervalle.
eval_interval = 8_192            # Trainingsschritte pro Intervall
total_timesteps = 200_000       # Gesamtzahl der Trainingsschritte
n_iterations = total_timesteps // eval_interval

# Listen für Plot-Daten
train_loss_list = []  # Wir definieren Loss als negativen Reward (damit "kleiner" besser ist)
test_loss_list = []
timesteps_list = []

In [ ]:
import time

timestamps = 250_000
print("Modell läuft auf:", model.device)

# Debug-Print, um den Typ von model.policy zu prüfen
print(f"Typ von model.policy: {type(model.policy)}")

# Modell auf GPU setzen, ohne model.policy zu ersetzen
model.policy.to("cuda")
# Nur die forward()-Methode kompilieren, nicht das gesamte Policy-Objekt überschreiben
model.policy.forward = torch.compile(model.policy.forward)

# Prüfen, ob das Modell wirklich auf CUDA ist (Policy-Parameter verwenden)
print("Modell auf Gerät:", next(model.policy.parameters()).device)

# Teste, ob Stable-Baselines3 wirklich Tensoren auf der GPU erstellt
test_tensor = torch.randn(10, 10).to("cuda")
print("GPU-Test-Tensor erstellt:", test_tensor.device)

# Training um 'timestamps' Timesteps
print("Training beginnt.")
start_time = time.time()

#with torch.amp.autocast("cuda"):
model.learn(total_timesteps=timestamps, reset_num_timesteps=False)

end_time = time.time()
duration = end_time - start_time
duration_minutes = duration / 60.0
print(f"Training abgeschlossen. {timestamps} Timesteps haben {duration:.2f} Sekunden ({duration_minutes:.2f} Minuten) gedauert.")

In [ ]:
# Backtest-Train-Umgebung für Evaluation (ohne SubprocVecEnv, da wir nur eine Instanz brauchen)
train_backtest_env = TradingEnv(
    data=train_data,  # Oder test_data, falls du separate Test-Daten hast
    initial_cash=10_000,
    window_size=336,
    scaler_path="/home/dhbw/scaler.pkl",
    default_seed=SEED
)

In [ ]:
obs, info = train_backtest_env.reset(seed=SEED)
done = False

# Liste der actionen
action_list = []

while not done:
#for i in range(1000):
    # Bestimme die Aktion (deterministisch)
    action, _states = model.predict(obs, deterministic=True)
    action = int(action)  # oder: action = action.item()
    obs, reward, done, truncated, info = train_backtest_env.step(action)
    action_list.append(action)

# Hier wird der Zustand gerendert (z.B. als Plot). Du kannst den Render-Modus anpassen.
train_backtest_env.render(mode='human')
print(action_list)

In [ ]:
obs, info = test_env.reset(seed=SEED)
done = False

# Liste der actionen
action_list = []

while not done:
#for i in range(10):
    # Bestimme die Aktion (deterministisch)
    action, _states = model.predict(obs, deterministic=True)
    action = int(action)  # oder: action = action.item()
    obs, reward, done, truncated, info = test_env.step(action)
    action_list.append(action)

# Hier wird der Zustand gerendert (z.B. als Plot). Du kannst den Render-Modus anpassen.
test_env.render(mode='human')
print(action_list)

In [ ]:
import time

# Trainingsschleife in Intervallen
for i in range(1, n_iterations + 1):
    print(f"\n=== Trainingsiteration {i} von {n_iterations} ===")
    print("Modell läuft auf:", model.device)

    # Debug-Print, um den Typ von model.policy zu prüfen
    print(f"Typ von model.policy: {type(model.policy)}")

    # Modell auf GPU setzen, ohne model.policy zu ersetzen
    model.policy.to("cuda")
    # Nur die forward()-Methode kompilieren, nicht das gesamte Policy-Objekt überschreiben
    model.policy.forward = torch.compile(model.policy.forward)

    # Prüfen, ob das Modell wirklich auf CUDA ist (Policy-Parameter verwenden)
    print("Modell auf Gerät:", next(model.policy.parameters()).device)

    # Teste, ob Stable-Baselines3 wirklich Tensoren auf der GPU erstellt
    test_tensor = torch.randn(10, 10).to("cuda")
    print("GPU-Test-Tensor erstellt:", test_tensor.device)

    # Training um 'eval_interval' Timesteps
    print("Training beginnt.")
    start_time = time.time()
    model.learn(total_timesteps=eval_interval, reset_num_timesteps=False)
    end_time = time.time()
    duration = end_time - start_time
    duration_minutes = duration / 60.0
    print(f"Training abgeschlossen. {i * eval_interval} Timesteps haben {duration:.2f} Sekunden ({duration_minutes:.2f} Minuten) gedauert.")

    # Evaluation auf dem Trainings-Environment (mittlere Reward über 5 Episoden)
    with torch.no_grad():
        # Stelle sicher, dass die Policy auf GPU ist
        model.policy.to("cuda")
        mean_train_reward, _ = evaluate_policy(model, env, n_eval_episodes=5, deterministic=True, render=False)

    # Evaluation auf dem Test-Environment
    with torch.no_grad():
        model.policy.to("cuda")
        mean_test_reward, _ = evaluate_policy(model, test_env, n_eval_episodes=5, deterministic=True, render=False)
    
    # Um den "Loss" zu erhalten, verwenden wir den negativen Reward.
    train_loss = -mean_train_reward
    test_loss = -mean_test_reward
    
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    timesteps_list.append(i * eval_interval)
    
    print(f"Timesteps: {i * eval_interval} | Train Reward: {mean_train_reward:.2f} (Loss: {train_loss:.2f}) | Test Reward: {mean_test_reward:.2f} (Loss: {test_loss:.2f})")

In [ ]:
# -------------------------------
# Plot: Loss-Kurven (grün: Training, rot: Test)
# -------------------------------
plt.figure(figsize=(10, 6))
plt.plot(timesteps_list, train_loss_list, color='green', label='Train Loss')
plt.plot(timesteps_list, test_loss_list, color='red', label='Test Loss')
plt.xlabel("Timesteps")
plt.ylabel("Loss (negativer Reward)")
plt.title("Train vs. Test Loss Kurven")
plt.legend()
plt.grid()
plt.show()

# -------------------------------
# Trainiertes Modell speichern
# -------------------------------
model.save("ppo_trading_model")